In [1]:

from openai import OpenAI

client = OpenAI()

META_PROMPT = """
Given a task description or existing prompt, produce a detailed system prompt to guide a language model in completing the task effectively.

# Guidelines

- Understand the Task: Grasp the main objective, goals, requirements, constraints, and expected output.
- Minimal Changes: If an existing prompt is provided, improve it only if it's simple. For complex prompts, enhance clarity and add missing elements without altering the original structure.
- Reasoning Before Conclusions**: Encourage reasoning steps before any conclusions are reached. ATTENTION! If the user provides examples where the reasoning happens afterward, REVERSE the order! NEVER START EXAMPLES WITH CONCLUSIONS!
    - Reasoning Order: Call out reasoning portions of the prompt and conclusion parts (specific fields by name). For each, determine the ORDER in which this is done, and whether it needs to be reversed.
    - Conclusion, classifications, or results should ALWAYS appear last.
- Examples: Include high-quality examples if helpful, using placeholders [in brackets] for complex elements.
   - What kinds of examples may need to be included, how many, and whether they are complex enough to benefit from placeholders.
- Clarity and Conciseness: Use clear, specific language. Avoid unnecessary instructions or bland statements.
- Formatting: Use markdown features for readability. DO NOT USE ``` CODE BLOCKS UNLESS SPECIFICALLY REQUESTED.
- Preserve User Content: If the input task or prompt includes extensive guidelines or examples, preserve them entirely, or as closely as possible. If they are vague, consider breaking down into sub-steps. Keep any details, guidelines, examples, variables, or placeholders provided by the user.
- Constants: DO include constants in the prompt, as they are not susceptible to prompt injection. Such as guides, rubrics, and examples.
- Output Format: Explicitly the most appropriate output format, in detail. This should include length and syntax (e.g. short sentence, paragraph, JSON, etc.)
    - For tasks outputting well-defined or structured data (classification, JSON, etc.) bias toward outputting a JSON.
    - JSON should never be wrapped in code blocks (```) unless explicitly requested.

The final prompt you output should adhere to the following structure below. Do not include any additional commentary, only output the completed system prompt. SPECIFICALLY, do not include any additional messages at the start or end of the prompt. (e.g. no "---")

[Concise instruction describing the task - this should be the first line in the prompt, no section header]

[Additional details as needed.]

[Optional sections with headings or bullet points for detailed steps.]

# Steps [optional]

[optional: a detailed breakdown of the steps necessary to accomplish the task]

# Output Format

[Specifically call out how the output should be formatted, be it response length, structure e.g. JSON, markdown, etc]

# Examples [optional]

[Optional: 1-3 well-defined examples with placeholders if necessary. Clearly mark where examples start and end, and what the input and output are. User placeholders as necessary.]
[If the examples are shorter than what a realistic example is expected to be, make a reference with () explaining how real examples should be longer / shorter / different. AND USE PLACEHOLDERS! ]

# Notes [optional]

[optional: edge cases, details, and an area to call or repeat out specific important considerations]
""".strip()

def generate_prompt(task_or_prompt: str):
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": META_PROMPT,
            },
            {
                "role": "user",
                "content": "Task, Goal, or Current Prompt:\n" + task_or_prompt,
            },
        ],
    )

    return completion.choices[0].message.content

In [6]:
task = """
Task Description
You are implementing a multi-hop question answering system using three search tools to query a knowledge base of GSW entities. Your task is to decompose complex questions into sequential sub-queries, where each "hop" finds one piece of information needed for the next step.
CRITICAL REQUIREMENT: You must provide detailed reasoning with every single tool call explaining why you chose that specific tool and query.
Your Three Tools
Tool 1: search_gsw_bm25_entity_name
Use for: Direct entity name searches, especially for Hop 1 queries
Examples: "Christopher Nolan", "Billy Giles", "Napoleon"
Tool 2: search_gsw_bm25_entity_with_entity_features
Use for: Relationship-based queries, entities defined by their properties
Examples: "first President of Namibia", "mother of Marie Antoinette", "currency of Northern Ireland"
Tool 3: get_multiple_relevant_entity_contexts
Use for: MANDATORY after every search to get full entity information
Always required: Must be called after Tools 1 or 2 to retrieve complete context
Detailed Reasoning Examples
Example 1: Linear Chain (2-Hop)
Question: "Who succeeded the first President of Namibia?"
Step 1: search_gsw_bm25_entity_with_entity_features("first President of Namibia")
Reasoning: "I need to find an entity defined by the relationship 'first President of Namibia'. This is not a direct entity name but a role-based query, so I use Tool 2 to find the entity that holds this specific position."
Step 2: get_multiple_relevant_entity_contexts([entity_id_from_step1])
Reasoning: "Mandatory context retrieval to get full information about the first president. I need to read the context to find the president's name (Sam Nujoma) and look for succession information."
Step 3: search_gsw_bm25_entity_with_entity_features("successor of Sam Nujoma")
Reasoning: "Now I need to find who succeeded Sam Nujoma. This is again a relationship-based query ('successor of X'), so I use Tool 2 to find the entity defined by this succession relationship."
Step 4: get_multiple_relevant_entity_contexts([entity_id_from_step3])
Reasoning: "Mandatory context retrieval to get information about the successor and confirm this gives me the final answer (Hifikepunye Pohamba)."
Example 2: Location-Based Chain (3-Hop)
Question: "What currency is used where Billy Giles died?"
Step 1: search_gsw_bm25_entity_name("Billy Giles")
Reasoning: "I start with the known entity name 'Billy Giles'. This is a direct entity search, so I use Tool 1 to find the person by name."
Step 2: get_multiple_relevant_entity_contexts([billy_giles_id])
Reasoning: "Mandatory context retrieval to get Billy Giles's information. I need to scan the context for death location information to identify where he died."
Step 3: search_gsw_bm25_entity_name("Belfast Northern Ireland")
Reasoning: "From the context, I found Billy Giles died in Belfast. I search for Belfast with geographical context hint. I use Tool 1 because 'Belfast' is an entity name, adding 'Northern Ireland' as a disambiguation hint."
Step 4: get_multiple_relevant_entity_contexts([belfast_id])
Reasoning: "Mandatory context retrieval to get Belfast's information. I need to find what administrative region Belfast belongs to and look for currency information."
Step 5: search_gsw_bm25_entity_with_entity_features("currency of Northern Ireland")
Reasoning: "From Belfast's context, I identified it's in Northern Ireland. Now I need to find the currency used there. This is a relationship query ('currency of X'), so I use Tool 2."
Step 6: get_multiple_relevant_entity_contexts([currency_id])
Reasoning: "Mandatory context retrieval to get the currency information and confirm the final answer (pound sterling)."
Example 3: Complex Chain (4-Hop)
Question: "When did Napoleon occupy the city where the mother of the woman who brought Louis XVI style to the court died?"
Step 1: search_gsw_bm25_entity_with_entity_features("brought Louis XVI style to court")
Reasoning: "I need to find the woman who brought Louis XVI style to court. This is a relationship-based query describing what someone did, not a direct name, so I use Tool 2."
Step 2: get_multiple_relevant_entity_contexts([marie_antoinette_id])
Reasoning: "Mandatory context retrieval to get information about Marie Antoinette. I need to look for family relationship information, specifically about her mother."
Step 3: search_gsw_bm25_entity_with_entity_features("mother of Marie Antoinette")
Reasoning: "I need to find Marie Antoinette's mother. This is a family relationship query ('mother of X'), so I use Tool 2 to find the entity defined by this relationship."
Step 4: get_multiple_relevant_entity_contexts([maria_theresa_id])
Reasoning: "Mandatory context retrieval to get Maria Theresa's information. I need to find where she died to identify the city for the next hop."
Step 5: search_gsw_bm25_entity_name("Vienna")
Reasoning: "From Maria Theresa's context, I found she died in Vienna. Now I search for Vienna as a direct entity name using Tool 1."
Step 6: get_multiple_relevant_entity_contexts([vienna_id])
Reasoning: "Mandatory context retrieval to get Vienna's information and confirm it's the correct city. I may also find occupation events in the context."
Step 7: search_gsw_bm25_entity_with_entity_features("Napoleon occupy Vienna")
Reasoning: "I need to find when Napoleon occupied Vienna. This is an event-based relationship query connecting Napoleon to Vienna through an occupation event, so I use Tool 2."
Step 8: get_multiple_relevant_entity_contexts([occupation_event_id])
Reasoning: "Mandatory context retrieval to get the occupation event details and find the date (1805) for the final answer."
Key Reasoning Principles
Tool Selection Logic:

Known entity name → Tool 1 + reasoning about direct entity search
Relationship/role/property query → Tool 2 + reasoning about relationship type
After any search → Tool 3 + reasoning about what information to extract

Reasoning Requirements:

Explain tool choice: Why this specific tool for this query type
Explain query formulation: How you constructed the search terms
Explain next steps: What you're looking for in the context
Explain hop logic: How this step connects to the overall question

Context Analysis:

Information extraction: What specific data you found
Next hop planning: What additional information is needed
Entity identification: What entities to search for next

Remember: Every single tool call must include detailed reasoning explaining your decision-making process.

Output Format:
When you find the answer, respond with ONLY this JSON:
{
    "reasoning": "Step-by-step explanation with evidence for each hop",
    "answer": "Just the final answer in several words, no extra words"
}

Do NOT include phrases like "The answer is" or "Based on my search" in the answer field.

"""

In [7]:
print(generate_prompt(task_or_prompt=task))

Your task is to decompose complex questions into sequential sub-queries for a multi-hop question answering system, using three search tools to query a knowledge base of GSW entities. Each tool call must include detailed reasoning explaining why you chose that specific tool and query.

You have three tools at your disposal:

1. **search_gsw_bm25_entity_name**: Use this tool for direct entity name searches, most useful for initial ("Hop 1") queries.
2. **search_gsw_bm25_entity_with_entity_features**: Use this tool for queries based on entity relationships or properties.
3. **get_multiple_relevant_entity_contexts**: Mandatory after every search (Tools 1 or 2) to retrieve full entity information.

# Key Reasoning Principles

- **Tool Selection Logic**:
  - **Known entity name** → Use Tool 1 with reasoning for a direct entity search.
  - **Relationship/role/property query** → Use Tool 2 with reasoning on the relationship type.
  - After any search → Use Tool 3 with reasoning on what informa